# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size= "Humidity",
    c= "Humidity",
    hover_cols= ['City', 'Humidity'],
    alpha=0.7,
    title='City Humidity Map',
)

# Display the map plot
humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
humidity_threshold = 70

filtered_cities = city_data_df[city_data_df['Humidity'] > humidity_threshold]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
...,...,...,...,...,...,...,...,...,...,...
570,570,staryy nadym,65.6118,72.6842,4.57,90,99,8.05,RU,1666108561
571,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,1666108563
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities.copy()

{
    'City': ['City1', 'City2', 'City3'],
    'Country': ['Country1', 'Country2', 'Country3'],
    'Latitude': [40.7128, 34.0522, 41.8781],
    'Longitude': [-74.0060, -118.2437, -87.6298],
    'Humidity': [80, 75, 85],
    'Hotel Name' : []
}

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API


# Display sample data
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
...,...,...,...,...,...,...,...,...,...,...
570,570,staryy nadym,65.6118,72.6842,4.57,90,99,8.05,RU,1666108561
571,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,1666108563
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000
params = {
    'radius': radius,
    'type': 'lodging',
    'key': '256a72cdab2647119a83ce7feeed36f7',
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point:{longitude},{latitude}"
    params["bias"] = f"point:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
farsund - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
dukat - nearest hotel: No hotel found
senneterre - nearest hotel: No hotel found
grimmen - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
stornoway - nearest hotel: No hotel found
chokurdakh - nearest hotel: No hotel found
labuhan - nearest hotel: No hot

namibe - nearest hotel: No hotel found
kokkola - nearest hotel: No hotel found
leningradskiy - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
chegdomyn - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
bafoulabe - nearest hotel: No hotel found
kailua - nearest hotel: No hotel found
verkh-suetka - nearest hotel: No hotel found
dunedin - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
mahina - nearest hotel: No hotel found
kamloops - nearest hotel: No hotel found
krutikha - nearest hotel: No hotel found
bonavista - nearest hotel: No hotel found
zaysan - nearest hotel: No hotel found
nemuro - nearest hotel: No hotel found
saldanha - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
bengkulu - nearest hotel: No hotel found
honiara - nearest hotel: No hotel found
pemba - nearest hotel: No hotel found
ahipara - nearest hotel: No hotel found
shingu - neares

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228,No hotel found
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934,No hotel found
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231,No hotel found
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231,No hotel found
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
570,570,staryy nadym,65.6118,72.6842,4.57,90,99,8.05,RU,1666108561,No hotel found
571,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,1666108563,No hotel found
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564,No hotel found
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# Configure the map plot with additional information in the hover message
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    c="Humidity",
    hover_cols=['City', 'Country', 'Humidity'],  # Include 'Country' in the hover message
    alpha=0.7,
    title='City Humidity Map',
    label='Hotel Info',  # Add a label for the hover info
    datashade=True,  # Optional: Use datashader for large datasets
)

# Display the map plot
humidity_map


:DynamicMap   []
   :Overlay
      .Tiles.I        :Tiles   [x,y]
      .RGB.Hotel_Info :RGB   [Lng,Lat]   (R,G,B,A)